# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [75]:
#  Based on "school type" chart, the charter schools out-performed the public district schools across all metrics as 
#       indicated by the math scores, reading and overall passing rate.
#
#  Based on scores by "school size" chart, small and medium sized school performed better than large size schools as 
#       indicated by average math and reading score and overall passing rate.
#
#  Charter schools were the Top Performing Schools as indicated by the "Top Performing Schools" chart.
#
#  "Paul Bradley" 9th, Huang High School has student ID "0". 


### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [76]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()



,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [77]:
#Calculate the total number of schools
total_schools = school_data['School ID'].count()
total_schools

#Calculate the total number of students
total_students = student_data['Student ID'].count()
total_students

#Calculate the total budget
total_budget = school_data['budget'].sum()
total_budget

#Calculate the average math score 
average_math_score = student_data['math_score'].mean()
average_math_score

#Calculate the average reading score
average_reading_score = student_data['reading_score'].mean()
average_reading_score

#Calculate the overall passing rate (overall average score),
#i.e. (avg. math score + avg. reading score)/2

overall_passing_rate = (average_math_score + average_reading_score)/2
overall_passing_rate

#Calculate the percentage of students with a passing math score (70 or greater)

total_math_pass = len(school_data_complete[school_data_complete['math_score'] >= 70])
total_math_pass
math_pass_percentage = (total_math_pass/total_students)*100
math_pass_percentage


#Calculate the percentage of students with a passing reading score (70 or greater)

total_reading_pass = len(school_data_complete[school_data_complete['reading_score'] >= 70])
total_reading_pass
reading_pass_percentage = (total_reading_pass/total_students)*100
reading_pass_percentage

d={'Total Schools':[total_schools],
   'Total Students':[total_students],
   'Total Budget':[total_budget],
   'Average Math Score':[average_math_score],
   'Average Reading Score':[average_reading_score],
   '% Passing Math':[math_pass_percentage],
   '% Passing Reading':[reading_pass_percentage],
   '% Overall Passing Rate':[overall_passing_rate]
   }

df=pd.DataFrame(d)
df['Total Budget'] = df['Total Budget'].map("\${:,.2f}".format)
df['Average Math Score'] = df['Average Math Score'].map("{:,.2f}".format)
df['Average Reading Score'] = df['Average Reading Score'].map("{:,.2f}".format)
df['% Passing Math'] = df['% Passing Math'].map("{:,.2f}".format)
df['% Passing Reading'] = df['% Passing Reading'].map("{:,.2f}".format)
df['% Overall Passing Rate'] = df['% Overall Passing Rate'].map("{:,.2f}".format)
df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"\$24,649,428.00",78.99,81.88,74.98,85.81,80.43


In [78]:
merged_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

#groups by school
by_school = merged_df.set_index("school_name").groupby(["school_name"])

#school types
sch_types = school_data.set_index("school_name")["type"]

# total students by school
stu_per_sch = by_school['Student ID'].count()

# school budget
sch_budget = school_data.set_index("school_name")["budget"]

#school budget per per student
stu_budget = school_data.set_index("school_name")["budget"]/school_data.set_index("school_name")["size"]

#averageg scores by school
avg_math = by_school["math_score"].mean()
avg_read = by_school["reading_score"].mean()

# % passing scores
pass_math = merged_df[merged_df["math_score"] >= 70].groupby("school_name")['Student ID'].count()/stu_per_sch * 100
pass_read = merged_df[merged_df["reading_score"] >= 70].groupby("school_name")['Student ID'].count()/stu_per_sch * 100
overall = merged_df[(merged_df["reading_score"] >= 70) & (merged_df["math_score"] >= 70)].groupby("school_name")["Student ID"].count()/stu_per_sch * 100

sch_summary = pd.DataFrame({
    "School Type": sch_types,
    "Total Students": stu_per_sch,
    "Per Student Budget": stu_budget,
    "Total School Budget": sch_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
})


school_summary = sch_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          'Overall Passing Rate']]


school_summary.style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.0f}",
                          'Average Math Score': "{:.1f}", 
                          'Average Reading Score': "{:.1f}", 
                          "% Passing Math": "{:.1%}", 
                          "% Passing Reading": "{:.1%}", 
                          "Overall Passing Rate": "{:.1%}"})

merged_innerf_df = school_summary.copy()
merged_innerf_df




,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [79]:
#Sort and display the top five schools in overall passing rate
best_performing_schools = merged_innerf_df.sort_values("Overall Passing Rate", ascending = False)
best_performing_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [80]:
#Sort and display the five worst-performing schools
worst_performing_schools = merged_innerf_df.sort_values("Overall Passing Rate", ascending = True)
worst_performing_schools.head(5)



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [81]:
#Create list for 9th grade groupby school name
Math_Scores_9th = school_summary_df[school_summary_df['grade'] == '9th'].groupby('school_name')['math_score'].mean()

#Create list for 10th grade groupby school name
Math_Scores_10th = school_summary_df[school_summary_df['grade'] == '10th'].groupby('school_name')['math_score'].mean()

#Create list for 11th grade groupby school name
Math_Scores_11th = school_summary_df[school_summary_df['grade'] == '11th'].groupby('school_name')['math_score'].mean()

#Create list for 12th grade groupby school name
Math_Scores_12th = school_summary_df[school_summary_df['grade'] == '12th'].groupby('school_name')['math_score'].mean()

Math_Scores = pd.DataFrame({
        "9th": Math_Scores_9th,
        "10th": Math_Scores_10th,
        "11th": Math_Scores_11th,
        "12th": Math_Scores_12th
        })
Math_Scores
Math_Scores.style.format({'9th': '{:.2f}', 
                          "10th": '{:.2f}', 
                          "11th": "{:.2f}", 
                          "12th": "{:.2f}"})


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [82]:
#Create list for 9th grade groupby school name
Reading_Scores_9th = school_summary_df[school_summary_df['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
#Create list for 10th grade groupby school name
Reading_Scores_10th = school_summary_df[school_summary_df['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
#Create list for 11th grade groupby school name
Reading_Scores_11th = school_summary_df[school_summary_df['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
#Create list for 12th grade groupby school name
Reading_Scores_12th = school_summary_df[school_summary_df['grade'] == '12th'].groupby('school_name')['reading_score'].mean()


Reading_Scores = pd.DataFrame({
        "9th": Reading_Scores_9th,
        "10th": Reading_Scores_10th,
        "11th": Reading_Scores_11th,
        "12th": Reading_Scores_12th
        })

Reading_Scores
Reading_Scores.style.format({'9th': '{:.2f}', 
                             "10th": '{:.2f}', 
                             "11th": "{:.2f}", 
                             "12th": "{:.2f}"})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [83]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [84]:
merged_df = school_data_complete
merged_df['spending_bins'] = pd.cut(merged_df['budget']/merged_df['size'], spending_bins, labels = group_names)
merged_df['spending_bins']

#group by spending
group_by_spending = merged_df.groupby('spending_bins')

avg_math  = group_by_spending['math_score'].mean()
avg_read  = group_by_spending['reading_score'].mean()

a = merged_df[merged_df['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()
b = group_by_spending['Student ID'].size()


pass_math = merged_df[merged_df['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/group_by_spending['Student ID'].count()
pass_read = merged_df[merged_df['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/group_by_spending['Student ID'].count()
overall   = merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/group_by_spending['Student ID'].count()


scores_by_spend = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall          
})

scores_by_spend
scores_by_spend.style.format({'Average Math Score': '{:.2f}', 
                              'Average Reading Score': '{:.2f}', 
                              '% Passing Math': '{:.2%}', 
                              '% Passing Reading':'{:.2%}', 
                              'Overall Passing Rate': '{:.2%}'})



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
spending_bins,,,,,
<$585,83.36,83.96,93.70%,96.69%,90.64%
$585-615,83.53,83.84,94.12%,95.89%,90.12%
$615-645,78.06,81.43,71.40%,83.61%,60.29%
$645-675,77.05,81.01,66.23%,81.11%,53.53%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [85]:
# Sample bins. Feel free to create your own bins.
size_bins = [0,1000,2000,5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [86]:
merged_df = school_data_complete
merged_df['size_bins'] = pd.cut(merged_df['size'], size_bins, labels = group_names)


In [87]:
#group by spending
group_by_size = merged_df.groupby('size_bins')

In [88]:
avg_math = group_by_size['math_score'].mean()
avg_read = group_by_size['math_score'].mean()
pass_math = merged_df[merged_df['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/group_by_size['Student ID'].count()
pass_read = merged_df[merged_df['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/group_by_size['Student ID'].count()
overall = merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/group_by_size['Student ID'].count()


# df build            
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})
scores_by_size
scores_by_size.style.format({'Average Math Score': '{:.2f}', 
                              'Average Reading Score': '{:.2f}', 
                              '% Passing Math': '{:.2%}', 
                              '% Passing Reading':'{:.2%}', 
                              'Overall Passing Rate': '{:.2%}'})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
size_bins,,,,,
Small (<1000),83.83,83.83,93.95%,96.04%,90.14%
Medium (1000-2000),83.37,83.37,93.62%,96.77%,90.62%
Large (2000-5000),77.48,77.48,68.65%,82.13%,56.57%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [73]:
merged_df = school_data_complete
group_by_type = merged_df.groupby("type")

In [74]:
avg_math = group_by_type['math_score'].mean()
avg_read = group_by_type['math_score'].mean()
pass_math = merged_df[merged_df['math_score'] >= 70].groupby('type')['Student ID'].count()/group_by_type['Student ID'].count()
pass_read = merged_df[merged_df['reading_score'] >= 70].groupby('type')['Student ID'].count()/group_by_type['Student ID'].count()
overall = merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('type')['Student ID'].count()/group_by_type['Student ID'].count()

scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall})

scores_by_type
scores_by_type.style.format({'Average Math Score': '{:.2f}', 
                              'Average Reading Score': '{:.2f}', 
                              '% Passing Math': '{:.2%}', 
                              '% Passing Reading':'{:.2%}', 
                              'Overall Passing Rate': '{:.2%}'})



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,
Charter,83.41,83.41,93.70%,96.65%,90.56%
District,76.99,76.99,66.52%,80.91%,53.70%
